# PHYS20762 - Project - 

David Phelan  
University of Manchester  
March 2025

This project aims to simulate a driven, damped harmonic oscillator.

In [87]:
# Initilisation
import numpy as np
import matplotlib.pyplot as plt

The below function is an Euler solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [88]:
def Euler_undriven(m, k, b, n, x0, v0, dt):
    """
    Forward Euler method for the undriven (possibly damped) oscillator:
        m x''(t) + b x'(t) + k x(t) = 0

    Args:
        m   (float): Mass of the oscillator.
        k   (float): Spring constant.
        b   (float): Damping coefficient (0 for no damping).
        n   (int)  : Number of time steps.
        x0  (float): Initial position x(0).
        v0  (float): Initial velocity x'(0).
        dt  (float): Timestep size.

    Returns:
        x (ndarray): Positions at each timestep (length n+1).
        v (ndarray): Velocities at each timestep (length n+1).
        a (ndarray): Accelerations at each timestep (length n+1).
        t (ndarray): Array of time points from 0 to n*dt (length n+1).

    Notes:
        - This function implements the basic "forward Euler" scheme.
        - The acceleration is computed via a[i+1] = -(k*x[i+1] + b*v[i+1]) / m
          before x[i+1], v[i+1] are updated, matching your original code flow.
        - For purely undamped motion, pass b=0.
    """
    
    time = np.linspace(0, n*dt, n+1)
    
    # Allocate arrays for position, velocity, and acceleration
    position = np.zeros(n+1)
    velocity = np.zeros(n+1)
    acceleration = np.zeros(n+1)
    
    # Initial conditions
    position[0] = x0
    velocity[0] = v0
    acceleration[0] = -(k * position[0] + b * velocity[0]) / m # Compute initial acceleration at t=0
    
    # Forward Euler time-stepping
    for i in range(n):
        # Compute acceleration for the next index (as in your original code)
        acceleration[i+1] = -(k * position[i+1] + b * velocity[i+1]) / m
        
        # Update velocity using the old acceleration
        velocity[i+1] = velocity[i] + acceleration[i] * dt
        
        # Update position using the old velocity
        position[i+1] = position[i] + velocity[i] * dt
    
    return position, velocity, acceleration, time


Below is code to verify that the Euler function is performing as expected, by checking generated values with expected values using the python keyword assert.

In [89]:
def test_Euler_undriven():
    """
    Tests the Euler_undriven function in simple edge cases
    to confirm that the solver behaves as expected.
    
    Scenario 1: 
        - m=1, k=0, b=0, x0=0, v0=0, dt=0.1
        - Expect no forces and no initial motion => everything remains zero.
    
    Scenario 2:
        - m=1, k=0, b=0, x0=2, v0=-1, dt=0.1
        - Expect velocity remains constant at -1, 
          position decreases linearly with slope -1, 
          acceleration is always zero.
    """
    # -- Scenario 1: no forces and no initial velocity --
    from_module = Euler_undriven(m=1, k=0, b=0, x0=0, v0=0, dt=0.1, n=10)
    x_arr, v_arr, a_arr, t_arr = from_module
    
    # Our expected arrays: everything should be zero (length n_stneps+1 = 11)
    zeros_11 = np.zeros(11)
    
    # Assert they match (within floating-point tolerance)
    assert np.allclose(x_arr, zeros_11), "Scenario 1: x should remain zero."
    assert np.allclose(v_arr, zeros_11), "Scenario 1: v should remain zero."
    assert np.allclose(a_arr, zeros_11), "Scenario 1: a should remain zero."
    
    # -- Scenario 2: constant velocity, no forces --
    from_module = Euler_undriven(m=1, k=0, b=0, x0=2, v0=-1, dt=0.1, n=10)
    x_arr, v_arr, a_arr, t_arr = from_module
    
    # Time array for reference
    # t_arr = [0.0, 0.1, 0.2, ..., 1.0] (length = 11)
    
    # The velocity should remain at -1:
    expected_v = -1 * np.ones_like(t_arr)
    # The position should be x(t) = 2 + v0 * t = 2 - 1 * t
    expected_x = 2 - t_arr
    # Acceleration should be zero (no spring, no damping)
    expected_a = np.zeros_like(t_arr)
    
    assert np.allclose(v_arr, expected_v),  "Scenario 2: velocity should stay at -1."
    assert np.allclose(x_arr, expected_x),  "Scenario 2: position should be linear in time."
    assert np.allclose(a_arr, expected_a),  "Scenario 2: acceleration should remain zero."
    
    print("All Euler_undriven tests passed successfully!")

test_Euler_undriven()

All Euler_undriven tests passed successfully!


The below function is an improved Euler solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [90]:
def ImprovedEuler_undriven(m, k, b, n, x0, v0, dt):
    """
    Implements the 'Improved Euler' approach shown in the slide:
        x_{i+1} = x_i + h * v_i + 0.5 * h^2 * a_i
        v_{i+1} = v_i + h * a_i
        a_i     = -(k/m)*x_i - (b/m)*v_i

    Args:
        m   : mass
        k   : spring constant
        b   : damping coefficient
        n   : number of time steps
        x0  : initial position
        v0  : initial velocity
        dt  : time step

    Returns:
        x, v, a, t
          x, v, a are arrays of length (n+1);
          t is the array of time points from 0 to n*dt.
    """

    time = np.linspace(0, n*dt, n+1)
    
    position = np.zeros(n+1)
    velocity = np.zeros(n+1)
    acceleration = np.zeros(n+1)

    # Initial conditions
    position[0] = x0
    velocity[0] = v0
    acceleration[0] = -(k*position[0] + b*velocity[0]) / m  # acceleration at i=0

    for i in range(n):
        # Position update includes a half-step for the acceleration
        position[i+1] = position[i] + dt*velocity[i] + 0.5*(dt**2)*acceleration[i]
        
        # Velocity update is the standard Euler step
        velocity[i+1] = velocity[i] + dt*acceleration[i]

        # Compute the new acceleration at i+1
        acceleration[i+1] = -(k*position[i+1] + b*velocity[i+1]) / m

    return position, velocity, acceleration, time

Below is code to verify that the improved Euler function is performing as expected, by checking generated values with expected values using the python keyword assert.

In [91]:
def test_ImprovedEuler_undriven():
    """
    Tests the Improved Euler (2nd-order Taylor) method in simple edge cases:
    
    Scenario 1: 
        k=0, b=0, v0=0 => no force, no initial velocity
        Expected: The mass stays at the initial position, with zero velocity/acceleration.

    Scenario 2: 
        k=0, b=0, v0 != 0 => no force, nonzero initial velocity
        Expected: The mass moves with constant velocity (v0) 
                  so x(t) = x0 + v0 * t, and a(t) = 0.
    """
    # -----------------------------
    # Scenario 1: No force, no initial motion
    # -----------------------------
    from_func = ImprovedEuler_undriven(m=1, k=0, b=0, n=10, x0=2, v0=0, dt=0.1)
    x_arr, v_arr, a_arr, t_arr = from_func
    
    # All values should remain exactly the initial position, 
    # with zero velocity and zero acceleration.
    expected_x = np.full_like(t_arr, 2.0)  # always at x=2
    expected_v = np.zeros_like(t_arr)
    expected_a = np.zeros_like(t_arr)
    
    assert np.allclose(x_arr, expected_x), "Scenario 1: position should remain constant at 2."
    assert np.allclose(v_arr, expected_v), "Scenario 1: velocity should remain zero."
    assert np.allclose(a_arr, expected_a), "Scenario 1: acceleration should remain zero."

    # -----------------------------
    # Scenario 2: No force, but nonzero initial velocity
    # -----------------------------
    from_func = ImprovedEuler_undriven(m=1, k=0, b=0, n=10, x0=2, v0=-1, dt=0.1)
    x_arr, v_arr, a_arr, t_arr = from_func
    
    # Velocity should stay at -1, position should decrease linearly
    # x(t) = x0 + v0 * t = 2 - t
    expected_v = -1.0 * np.ones_like(t_arr)
    expected_x = 2.0 + (-1.0)*t_arr
    expected_a = np.zeros_like(t_arr)
    
    assert np.allclose(v_arr, expected_v),  "Scenario 2: velocity should remain constant at -1."
    assert np.allclose(x_arr, expected_x),  "Scenario 2: position should be 2 - t."
    assert np.allclose(a_arr, expected_a),  "Scenario 2: acceleration should remain zero."

    print("All tests for the Improved Euler (2nd-order) method passed successfully!")

test_ImprovedEuler_undriven()

All tests for the Improved Euler (2nd-order) method passed successfully!


The below function is an Euler-Cromer solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [92]:
def EulerCromer_undriven(m, k, b, n, x0, v0, dt):
    """
    Euler–Cromer method for the undriven (possibly damped) oscillator:
        m x'' + b x' + k x = 0

    Updates:
        v_{i+1} = v_i + dt * a_i
        x_{i+1} = x_i + dt * v_{i+1}
    where
        a_i = -(k*x_i + b*v_i) / m.

    Args:
        m   (float): Mass of the oscillator
        k   (float): Spring constant
        b   (float): Damping coefficient (0 for no damping)
        n   (int)  : Number of time steps
        x0  (float): Initial position (x(0))
        v0  (float): Initial velocity (x'(0))
        dt  (float): Timestep size

    Returns:
        x (ndarray): Positions at each timestep (length n+1)
        v (ndarray): Velocities at each timestep (length n+1)
        a (ndarray): Accelerations at each timestep (length n+1)
        t (ndarray): Time points from 0 to n*dt (length n+1)
    """
    
    time = np.linspace(0, n*dt, n+1)
   
    position = np.zeros(n+1)
    velocity = np.zeros(n+1)
    acceleration = np.zeros(n+1)

    # Initial conditions
    position[0] = x0
    velocity[0] = v0
    acceleration[0] = -(k*x0 + b*v0) / m

    # Euler–Cromer loop
    for i in range(n):
        # Update velocity using the old acceleration
        velocity[i+1] = velocity[i] + dt*acceleration[i]
        # Update position using the new velocity
        position[i+1] = position[i] + dt*velocity[i+1]
        # Compute the new acceleration at i+1
        acceleration[i+1] = -(k*position[i+1] + b*velocity[i+1]) / m

    return position, velocity, acceleration, time

Below is code to verify that the Euler-Cromer function is performing as expected, by checking generated values with expected values using the python keyword assert.

In [93]:
def test_EulerCromer_undriven():
    """
    Tests the Euler-Cromer method in simple edge cases:
    
    1) No force, no initial velocity => the mass should remain at rest.
    2) No force, nonzero initial velocity => the mass moves at constant velocity.
    """
    # --- Scenario 1: No force, no motion ---
    x_arr, v_arr, a_arr, t_arr = EulerCromer_undriven(
        m=1, k=0, b=0, n=10, x0=5, v0=0, dt=0.1
    )
    # Expect position=constant=5, velocity=0, acceleration=0
    expected_x = np.full_like(t_arr, 5.0)
    expected_v = np.zeros_like(t_arr)
    expected_a = np.zeros_like(t_arr)

    assert np.allclose(x_arr, expected_x), "Scenario 1: x should remain at 5."
    assert np.allclose(v_arr, expected_v), "Scenario 1: v should remain 0."
    assert np.allclose(a_arr, expected_a), "Scenario 1: a should remain 0."

    # --- Scenario 2: No force, constant velocity ---
    x_arr, v_arr, a_arr, t_arr = EulerCromer_undriven(
        m=1, k=0, b=0, n=10, x0=5, v0=2, dt=0.1
    )
    # Velocity=2, so x(t)=5 + 2*t, a(t)=0
    expected_v = np.full_like(t_arr, 2.0)
    expected_x = 5 + 2*t_arr
    expected_a = np.zeros_like(t_arr)

    assert np.allclose(v_arr, expected_v), "Scenario 2: velocity should remain 2."
    assert np.allclose(x_arr, expected_x), "Scenario 2: position should be 5 + 2*t."
    assert np.allclose(a_arr, expected_a), "Scenario 2: acceleration should remain 0."

    print("All Euler-Cromer tests passed successfully!")

test_EulerCromer_undriven()

All Euler-Cromer tests passed successfully!


Vernet

In [94]:
def Verlet_undriven(m, k, b, n, x0, v0, dt):
    """
    Two-step method for the undriven (possibly damped) oscillator:
        m x'' + b x' + k x = 0
    using the recurrence relation:
        x_{i+1} = A x_i + B x_{i-1},
    with A, B as in the question. The first step x[1] is computed by
    an improved-Euler-style update for consistency.

    Args:
        m   : Mass of the oscillator
        k   : Spring constant
        b   : Damping coefficient (0 => no damping)
        n   : Number of time steps
        x0  : Initial position, x(0)
        v0  : Initial velocity, x'(0)
        dt  : Timestep size, h

    Returns:
        x, v, a, t
          x (ndarray): positions at each timestep (length n+1)
          v (ndarray): velocities approximated by finite difference (length n+1)
          a (ndarray): accelerations computed from x and v (length n+1)
          t (ndarray): time points from 0..n*dt (length n+1)
    """

    # Precompute constants for the two-step recurrence
    D = 2*m + b*dt
    A = 2.0*(2*m - k*(dt**2)) / D
    B = (b*dt - 2*m) / D

    time = np.linspace(0, n*dt, n+1)

    position = np.zeros(n+1)
    velocity = np.zeros(n+1)
    acceleration = np.zeros(n+1)

    # Initial conditions
    position[0] = x0
    velocity[0] = v0
    # Acceleration at t=0
    acceleration[0] = -(k*x0 + b*v0)/m

    # -----------------------------------------------------------
    # KICK-START STEP (i=0 --> i=1) using "Improved Euler" (2nd order)
    #
    # x1 = x0 + dt*v0 + 0.5*dt^2 * a0
    # v1 = v0 + dt*a0
    # -----------------------------------------------------------
    position[1] = position[0] + dt*velocity[0] + 0.5*(dt**2)*acceleration[0]
    velocity[1] = velocity[0] + dt*acceleration[0]
    acceleration[1] = -(k*position[1] + b*velocity[1]) / m

    # -----------------------------------------------------------
    # MAIN LOOP: Two-step recurrence for x_{i+1},
    # then derive v_{i+1} and a_{i+1} for consistency
    # -----------------------------------------------------------
    for i in range(1, n):
        # Position update from the two-step formula
        position[i+1] = A*position[i] + B*position[i-1]

        # Approximate velocity via forward difference
        velocity[i+1] = (position[i+1] - position[i]) / dt

        # Compute acceleration from the ODE definition
        acceleration[i+1] = -(k*position[i+1] + b*velocity[i+1]) / m

    return position, velocity, acceleration, time


In [95]:
def test_Verlet_undriven():
    # Case 1: No force, no initial velocity => remain at rest, x=constant
    x_arr, v_arr, a_arr, t_arr = Verlet_undriven(m=1, k=0, b=0, n=10, x0=2, v0=0, dt=0.1)
    assert np.allclose(x_arr, 2), "Position should remain at x=2."
    assert np.allclose(v_arr, 0), "Velocity should stay zero."
    assert np.allclose(a_arr, 0), "Acceleration should be zero."

    # Case 2: No force, constant nonzero velocity => x(t)=x0 + v0*t
    x_arr, v_arr, a_arr, t_arr = Verlet_undriven(m=1, k=0, b=0, n=10, x0=3, v0=1, dt=0.1)
    expected_x = 3 + t_arr*1.0
    assert np.allclose(x_arr, expected_x), "Position should be x0 + v0*t."
    assert np.allclose(v_arr, 1.0), "Velocity should remain constant at 1."
    assert np.allclose(a_arr, 0), "Acceleration should remain zero."
    
    print("All Verlet_undriven tests passed!")

test_Verlet_undriven()

All Verlet_undriven tests passed!
